In [25]:
import json
from uuid import uuid4
from copy import deepcopy

n_detectors = 3
windDirection = 54 #degrees CW from North
windSpeed = 10 #ft/s
weaponsYield = 10 #kT
weaponsHeight = 0 #ft
weaponLat = 41.270142 #degrees
weaponLon = -74.624634 #degrees

# Generate random UUIDs
id_value = str(uuid4())
metadata_key = str(uuid4())
owner_id = "081108a1-a245-42e3-9fd5-428ce3d44c7f"

# Load calculations data from external file
with open("calcData.json", "r") as calculations_file:
  calculations_data = json.load(calculations_file)

# Generate random UUID for calculations data ID
calculations_data["calculations"][0]["id"] = str(uuid4())
# set the weapons data
# Update weapons in the calculations data
calculations_data["calculations"][0]["weapons"][0]["id"] = str(uuid4())  # Generate a new UUID for the weapon
calculations_data["calculations"][0]["weapons"][0]["detonationTime"] = "2023-11-18T18:00:00.000Z"  # Update detonation time
calculations_data["calculations"][0]["weapons"][0]["yield"] = weaponsYield  # Update yield
calculations_data["calculations"][0]["weapons"][0]["location"] = {"latitude": weaponLat, "longitude": weaponLon, "altitude": weaponsHeight}

# Load entities data from external file
with open("entities.json", "r") as entities_file:
    entities_data = json.load(entities_file)

# Create the JSON structure
data = {
  "id": id_value,
  "name": "uploadTest",
  "classification": "CUI",
  "metadata": [],
  "owner": {
    "name": "David G. Fobar",
    "id": owner_id
  },
  "calculations": calculations_data["calculations"],
  "entities": [],
  "weather": {
    "windDirection": windDirection,
    "windSpeed": windSpeed,
    "windSpeedUnits": "ft/s",
    "historicalDataMonth": ""
  },
  "errors": []
}

# Add items to the "metadata" field dynamically
for i in range(n_detectors): 

  lat = weaponLat+i
  lon = weaponLon+i
  new_metadata_key = str(uuid4())
  new_metadata_item = {"key": new_metadata_key, "value": '{"UIC":"BlueTarget","ParentUIC":"","Name":"Target","Personnel":[],"Team":"Blue","Service":"Stationary","SIDC":"Target-B","TargetType":"In the Open","DamageLevel":"Immediate Transient Incapacitation","TypeName":"Target","StrokeWidth":10,"SymbolColor":"Blue","CustomIcon":true}'}
  data["metadata"].append(new_metadata_item)

  # For each metadata item, add a corresponding entity with a specified location
  new_entity_location = {"latitude": lat, "longitude": lon}  # Specify your location here
  new_entity = deepcopy(entities_data["entities"][0])  # Copy the structure of the first entity
  new_entity["id"] = new_metadata_key
  new_entity["currentLocation"] = new_entity_location
  new_entity["path"]["timeLocation"][0]["location"] = new_entity_location
  data["entities"].append(new_entity)

# Merge with calculations data
#data.update(calculations_data)

# Convert the Python data structure to JSON
json_data = json.dumps(data, indent=2)

# Save the JSON data to a file
with open("uploadTest.json", "w") as json_file:
  json_file.write(json_data)